In [ ]:
def read_fasta(file_path):
    sequences = {}
    current_id = None
    current_sequence_lines = []
    VALID_DNA_CHARS = {'A', 'C', 'T', 'G', 'N'}

    try:
        # Step 1: Open the file for reading
        with open(file_path, 'r') as f:
            line_number = 0
            
            # Use a simple 'for' loop to iterate over lines
            for line in f:
                line_number += 1
                line = line.strip() # Remove any leading/trailing whitespace
                
                # Step 2: Check if the line is a header (starts with '>')
                if line.startswith('>'):
                    # Save the previously collected sequence before starting a new one
                    if current_id:
                        sequences[current_id] = "".join(current_sequence_lines)
                        
                    # Extract the new ID
                    current_id = line[1:].strip()
                    current_sequence_lines = []
                
                # Step 3: Handle sequence lines
                elif current_id:
                    cleaned_line = line.upper()
                    
                    # SIMPLIFICATION: We replace the nested 'for' loop 
                    # with a single, powerful 'if' check using set logic.
                    
                    # Check if the set of characters in the line is NOT a subset of VALID_DNA_CHARS
                    if not set(cleaned_line).issubset(VALID_DNA_CHARS):
                        
                        # Find the specific invalid character(s)
                        invalid_chars = set(cleaned_line) - VALID_DNA_CHARS
                        # Just grab the first one to report in the error message
                        first_invalid_char = next(iter(invalid_chars)) 
                        
                        raise ValueError(
                            f"Invalid character '{first_invalid_char}' found in sequence {current_id} on line {line_number}. Only A, C, T, G, N are allowed."
                        )
                            
                    current_sequence_lines.append(cleaned_line)
                    
        # Step 4: Process the last sequence after the loop finishes
        if current_id and current_sequence_lines:
            sequences[current_id] = "".join(current_sequence_lines)
            
    # Step 5: Catch specific file errors (try/except)
    except FileNotFoundError:
        print(f"Error: Input file not found at '{file_path}'", file=sys.stderr)
        return None
    except ValueError as e:
        print(f"Error in file content: {e}", file=sys.stderr)
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}", file=sys.stderr)
        return None
    
    # Final check for empty input
    if not sequences:
        print("Error: No sequences found in the input file.", file=sys.stderr)
        return None
        
    return sequences

def translate_dna(dna_sequence, codon_table):
    """
    Translates a cleaned DNA sequence into an amino acid sequence using a simple while loop.
    
    Args:
        dna_sequence (str): The cleaned DNA sequence (uppercase, ACTGN).
        codon_table (dict): The standard genetic code.
        
    Returns:
        str: The translated amino acid sequence.
    """
    aa_sequence = []
    sequence_length = len(dna_sequence)
    
    # Initialize the index counter for the while loop
    i = 0
    
    # Use a simple 'while' loop to iterate through the sequence 3 bases at a time
    while i < sequence_length - 2:
        codon = dna_sequence[i:i+3]
        
        # Condition 1: Check for 'N' (unknown base)
        if 'N' in codon:
            aa_sequence.append('X')
        # Condition 2: Otherwise, try to translate using the genetic code
        else:
            # .get() handles any unexpected codons by defaulting to 'X' (unknown)
            amino_acid = codon_table.get(codon, 'X')
            aa_sequence.append(amino_acid)

        i += 3 # Move to the next codon
        
    return "".join(aa_sequence)


def main():
    """
    Main function to handle command-line arguments and orchestrate the translation.
    """
    # Step 1: Handle command-line arguments (sys)
    if len(sys.argv) < 2:
        print(f"Usage: python {os.path.basename(sys.argv[0])} <input_DNA_fasta> [output_AA_fasta]", file=sys.stderr)
        sys.exit(1)

    input_file = sys.argv[1]
    output_file = sys.argv[2] if len(sys.argv) > 2 else "translated_aa.faa"

    print(f"Starting DNA translation from '{input_file}' to '{output_file}'...")
    
    # Step 2: Read the input FASTA file
    dna_sequences = read_fasta(input_file)
    
    if dna_sequences is None:
        sys.exit(1)
        
    translated_sequences = {}
    
    # Step 3: Translate all sequences using a simple 'for' loop
    for seq_id, dna_seq in dna_sequences.items():
        if len(dna_seq) < 3:
            print(f"Warning: Sequence '{seq_id}' too short for translation (Length: {len(dna_seq)}). Skipping.", file=sys.stderr)
            continue
            
        aa_seq = translate_dna(dna_seq, GENETIC_CODE)
        translated_sequences[seq_id] = aa_seq
        
    # Step 4: Write the output to a file (try/except/finally)
    output_success = False

    try:
        # Using 'with open' for secure file writing
        with open(output_file, 'w') as output_f:
            # Use a simple 'for' loop to iterate through results
            for seq_id, aa_seq in translated_sequences.items():
                # Write FASTA header
                output_f.write(f">{seq_id}\n")
                
                # Write sequence, breaking lines (uses a simple while loop)
                i = 0
                while i < len(aa_seq):
                    output_f.write(aa_seq[i:i+80] + '\n')
                    i += 80
            output_success = True
            
    except Exception as e:
        print(f"\nCritical Error: Failed to write to output file '{output_file}'. Error: {e}", file=sys.stderr)
        
    finally:
        # The 'finally' block executes regardless of success or failure
        if output_success:
            print(f"Successfully wrote {len(translated_sequences)} translated sequence(s) to '{output_file}'.")
        else:
            print("Translation process completed with errors during file writing.", file=sys.stderr)


if __name__ == "__main__":
    main()
